In [72]:
import gym
from gym.spaces import Discrete
import numpy as np
import random

class deck:
   
    def __init__(self, randomize=True):
        self.suits = {"Diamonds": 0, "Clubs": 1, "Hearts":2, "Spades":3}
        self.values = {14:"Ace", 13:"King", 12:"Queen", 11:"Jack", 10:"10", 9:"9", 8:"8", 7:"7", 6:"6", 5:"5", 4:"4", 3:"3", 2:"2"}
        self.cards = np.array([])

        self.shuffle_deck(randomize)
    
    def shuffle_deck(self, randomize=True):
        self.cards = np.array([])
        for value in self.values.keys():
            for suit in self.suits.keys():
                self.cards = np.append(self.cards, card(suit, value))
        
        if randomize:
            np.random.shuffle(self.cards)

        return self.cards
    
    def __repr__(self):
        return f"{self.cards}"

    def draw(self):
        drawn_card = self.cards[0]
        self.cards = self.cards[1:]
        return drawn_card

class card:

    def __init__(self, suit, value):
        self.suit = suit
        self.value = value
        self.values = {14:"Ace", 13:"King", 12:"Queen", 11:"Jack", 10:"10", 9:"9", 8:"8", 7:"7", 6:"6", 5:"5", 4:"4", 3:"3", 2:"2"}
    
    def __eq__ (self, other):
        return (self.value == other.value)

    def __ne__ (self, other):
        return (self.value != other.value)

    def __lt__ (self, other):
        return (self.value < other.value)

    def __le__ (self, other):
        return (self.value <= other.value)

    def __gt__ (self, other):
        return (self.value > other.value)

    def __ge__ (self, other):
        return (self.value >= other.value)
    
    def __add__ (self, int):
        return card(self.suit, self.value+1)
    
    def __sub__ (self, int):
        return card(self.suit, self.value-1)
    
    def __repr__(self):
        return f'{self.values[self.value]} of {self.suit}'

class player:

    def __init__(self, name, agent=None):
        self.name = name
        self.hand = np.array([])
        self.agent = agent
        self.money = 0
        self.round_bet_already = 0
        self.status = "In"
        self.high = None
    
    def call(self, deck):
        self.money -= deck.call_amt

    def fold(self):
        self.status = False

    def raise_money(self, board, raise_amt):
        board.call_amt = raise_amt
        self.money -= board.call_amt
    
    def __repr__(self):
        return self.name

class PokerTable():

    def __init__(self, render_mode=None, budget=100, big_blind=2, small_blind=1):
        
        # Table Invariants
        self.num_players = 0
        self.players = []
        self.budget = budget
        self.big_blind = big_blind
        self.small_blind = small_blind
        self.render_mode = render_mode

        # Role Indexes
        self.dealer_position = None
        self.small_position = None
        self.big_position = None

        # Game Variants
        self.deck = None
        self.total_pot = 0
        self.round_pot = 0
        self.call_amt = 0
        self.last_raise = 0
        self.min_raise = 0
        self.last_player_raised = None
        self.table_cards = None
        self.player_order = None
        self.player_turn = 0

        # {0: Preflop, 1: Flop, 2: Turn, 3: River, 4: Showdown}
        self.stage = None

        # Rewards
        self.done = False
        self.illegal_moves_reward = -1
        self.round_start_funds = None
        self.round_end_funds = None
        self.fund_history = None 
        self.reward = None

        self.legal_moves = None
        # {0: Fold, 1: Check, 2: Call, 3: Min Raise, 4: 1.5x Min Raise, 5: 2x Min Raise, 6: 4x Min Raise, 7: 3x Big Blind, 8: 5x Big Blind, 9: All in}
        self.action_space = Discrete(5)
        self.observation = None

    def reset(self):
        self.obersvation = None
        self.reward = None 
        self.done = False
        self.fund_history = None

        for player in self.players:
            player.money = self.budget
    
    def step(self, action):
        pass
        
        
    def start_game(self):

        # Resets game variables
        self.table_cards = np.array([])
        self.deck = deck(randomize=True)
        self.stage = 0 #Preflop
        self.round_pot = 0
        self.total_pot = 0
        self.player_turn = 0
        self.round_pot = 0
        self.min_raise = 0
        self.call_amt = 0
        self.last_raise = 0
        self.last_player_raised = None
        
        # Shifts each position down to the next player
        self.dealer_position = self.dealer_position+1 if self.dealer_position+1 < self.num_players else 0
        self.small_position = self.dealer_position+1 if self.dealer_position+1 < self.num_players else 0
        self.big_position = self.small_position+1 if self.small_position+1 < self.num_players else 0

        # Resets players hands
        for player in self.players:
            player.hand = np.array([])
            player.status = "In"
        
        self.next_stage()
        self.next_player()
    
    def next_stage(self):
        
        self.stage += 1
        # Preflop
        if self.stage == 1:
            self.players[self.small_position].money -= self.small_blind
            self.players[self.big_position].money -= self.big_blind
            self.players[self.small_position].round_bet_already += self.small_blind
            self.players[self.big_position].round_bet_already += self.big_blind
            self.round_pot = self.small_blind + self.big_blind
            self.call_amt = self.big_blind
            self.min_raise = self.big_blind

            # Set up turn order
            if self.big_position+1 >= self.num_players:
                self.player_order = self.players
            else:
                self.player_order = self.players[self.big_position+1:] + self.players[:self.big_position+1] 

            # Deal hands
            for player in self.players:
                player.hand = np.append(player.hand, [self.deck.draw(), self.deck.draw()])
        else:
            self.player_turn = 0
            self.total_pot += self.round_pot
            self.round_pot = 0
            self.min_raise = 0
            self.call_amt = 0
            self.last_raise = 0
            self.last_player_raised = None

            for player in self.player_order:
                player.round_bet_already = 0

            if self.stage == 2:
                self.table_cards = np.append(self.table_cards, [self.deck.draw(), self.deck.draw(), self.deck.draw()])
            
            if self.stage == 3 or self.stage == 4:
                self.table_cards = np.append(self.table_cards, self.deck.draw())
        
        if self.stage == 5:
            self.eval_win()
        
        print(f"Stage{self.stage}: {self.table_cards}")
    
    def next_player(self):

        # if action goes back to the same person who raised
        if self.last_player_raised == self.player_order[self.player_turn]:
            self.next_stage()
        # if everyone checks 
        elif self.round_pot == 0 and self.player_turn == self.num_players-1:
            self.next_stage()
        # if it makes it back to first person without anyone raising in preflop
        elif self.stage == 1 and self.player_turn == 0 and self.last_player_raised == None and self.round_pot == self.num_players*self.big_blind:
            self.next_stage()

        if self.player_order[self.player_turn].status == "In":
            print(f"{self.player_order[self.player_turn]}'s turn")
            action = input(f"{self.player_order[self.player_turn]}'s turn")
            action = int(action)
            self.process_action(action)

        # Moves to the next player if the player is out or all-in   
        else:
            self.player_turn = self.player_turn + 1 if self.player_turn+1 != len(self.player_order) else 0
            self.next_player()
        

    def process_action(self, action):

        if self.check_legal(action):
            
            if action == 0: # Fold
                self.player_order[self.player_turn].status = "Out"
            # 1: Check -> do nothing
            elif action == 2: # Call
                self.player_order[self.player_turn].money -= self.call_amt - self.player_order[self.player_turn].round_bet_already
                self.round_pot += self.call_amt - self.player_order[self.player_turn].round_bet_already
                self.player_order[self.player_turn].round_bet_already += self.call_amt - self.player_order[self.player_turn].round_bet_already
            elif action in [3,4,5,6,7,8]: # Raises
                mult_dict = {3:1, 4:1.5, 5:2, 6:3, 7:4, 8:5}
                multiplier = mult_dict[action]

                self.player_order[self.player_turn].money -= self.call_amt + multiplier*self.min_raise - self.player_order[self.player_turn].round_bet_already
                self.player_order[self.player_turn].round_bet_already += self.call_amt + multiplier*self.min_raise - self.player_order[self.player_turn].round_bet_already
                # Minimum raise is twice the last raise amount
                self.round_pot += self.call_amt + multiplier*self.min_raise - self.player_order[self.player_turn].round_bet_already
                self.call_amt = self.call_amt + multiplier*self.min_raise
                self.min_raise *= 2*multiplier
                self.last_player_raised = self.player_order[self.player_turn]
                print(f"Call: ${self.call_amt}, Min Raise: {self.min_raise}")
            elif action == 9: # All in
                self.round_pot += self.player_order[self.player_turn].money
                self.player_order[self.player_turn].round_bet_already += self.player_order[self.player_turn].money
                if self.player_order[self.player_turn].money > self.min_raise + self.call_amt:
                    self.min_raise = 2*(self.player_order[self.player_turn].money - self.call_amt)
                if self.player_order[self.player_turn].money > self.call_amt:
                    self.call_amt = self.player_order[self.player_turn].money
                self.player_order[self.player_turn].money = 0
                self.last_player_raised = self.player_order[self.player_turn]
                self.player_order[self.player_turn].status = "All In"
                print(f"Call: ${self.call_amt}, Min Raise: {self.min_raise}")
                
            self.player_turn = self.player_turn + 1 if self.player_turn+1 != len(self.player_order) else 0
  
        else:
            # Does not increment player_turn by 1 if move is not legal
            print("Go again")
            pass
            #self.return_obs_illegal()
        
        #self.next_player()


    def get_obs(self):
        pass

    def eval_win(self):
        pass

    def check_legal(self, action):
        # {0: Fold, 1: Check, 2: Call, 3: Min Raise, 4: 1.5x Min Raise, 5: 2x Min Raise, 6: 4x Min Raise, 7: 3x Big Blind, 8: 5x Big Blind, 9: All in}

        active_player_money = self.player_order[self.player_turn].money
        # Fold and all-in are always legal moves
        legal_moves = [0, 9]
        # Check if check is legal
        if self.player_order[self.player_turn].round_bet_already == self.call_amt:
            legal_moves.append(1)
        # Check if call is legal
        if active_player_money > self.call_amt - self.player_order[self.player_turn].round_bet_already and self.call_amt - self.player_order[self.player_turn].round_bet_already != 0:
            legal_moves.append(2)
        # Check if raising by X amount is legal
        raise_dict = {1:3, 1.5:4, 2:5, 3:6, 4:7, 5:8}
        for mult in [1, 1.5, 2, 3, 4, 5]:
            if active_player_money >= self.call_amt + mult*self.min_raise:
                legal_moves.append(raise_dict[mult])
      

        return action in legal_moves
    
    def add_player(self, player):
        self.players.append(player)
        player.money = self.budget
        self.num_players += 1
        self.dealer_position = random.randint(0,self.num_players-1)
    
        

In [73]:
game = PokerTable()
Evob, Jakie, Bran, Keldeo = player("Evob"), player("Jakie"), player("Bran"), player("Keldeo")
game.add_player(Evob)
game.add_player(Jakie)
game.add_player(Bran)
game.add_player(Keldeo)

In [74]:
game.start_game()

Stage1: []
Evob's turn
Call: $100, Min Raise: 196


In [78]:
game.next_player()

Stage2: [3 of Clubs Ace of Hearts King of Clubs]
Stage3: [3 of Clubs Ace of Hearts King of Clubs 3 of Hearts]
Stage4: [3 of Clubs Ace of Hearts King of Clubs 3 of Hearts 6 of Spades]
Stage5: [3 of Clubs Ace of Hearts King of Clubs 3 of Hearts 6 of Spades]
Stage6: [3 of Clubs Ace of Hearts King of Clubs 3 of Hearts 6 of Spades]
Stage7: [3 of Clubs Ace of Hearts King of Clubs 3 of Hearts 6 of Spades]
Stage8: [3 of Clubs Ace of Hearts King of Clubs 3 of Hearts 6 of Spades]
Stage9: [3 of Clubs Ace of Hearts King of Clubs 3 of Hearts 6 of Spades]
Stage10: [3 of Clubs Ace of Hearts King of Clubs 3 of Hearts 6 of Spades]
Stage11: [3 of Clubs Ace of Hearts King of Clubs 3 of Hearts 6 of Spades]
Stage12: [3 of Clubs Ace of Hearts King of Clubs 3 of Hearts 6 of Spades]
Stage13: [3 of Clubs Ace of Hearts King of Clubs 3 of Hearts 6 of Spades]
Stage14: [3 of Clubs Ace of Hearts King of Clubs 3 of Hearts 6 of Spades]
Stage15: [3 of Clubs Ace of Hearts King of Clubs 3 of Hearts 6 of Spades]
Stage16:

RecursionError: maximum recursion depth exceeded while getting the repr of an object

In [68]:
print(Evob.money)
print(Jakie.money)
print(Bran.money)
print(Keldeo.money)
print(game.player_order)

0
0
0
0
[Bran, Keldeo, Evob, Jakie]


In [359]:
game.stage == 1 and game.player_turn == 0 and game.last_player_raised == None

True

In [70]:
game.player_turn

Jakie

In [82]:
# {0: Fold, 1: Check, 2: Call, 3: Min Raise, 4: 1.5x Min Raise, 5: 2x Min Raise, 6: 3x Min Raise, 7: 4x, 8: 5x, 9: All in}

2

In [153]:
game.player_order[0].money -= 2

In [151]:
game.player_turn

1

In [155]:
game.player_order

[Evob, Jakie, Bran, Keldeo]

In [243]:
np.all(np.array(list(map(lambda x: x.round_bet_already,[Evob, Jakie, Bran, Keldeo]))), 0) == 0

True

In [183]:
int(input())

2

In [239]:
np.all(np.array(list(map(lambda x: x.round_bet_already, [Evob, Jakie, Bran, Keldeo]))), True)

TypeError: an integer is required

In [80]:
for x in range(1):
    print(x)

0


In [82]:
np.array([(1,2), (2,3)])

array([[1, 2],
       [2, 3]])

In [85]:
np.array([(1, np.array([1,2])), (1, np.array([1,2]))])

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (2, 2) + inhomogeneous part.